# Script to prepare data for YOLO training

In [1]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Correctly specifying your file path
path_global_5s_train = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Global/train_global_recording_chunks.csv"
path_global_5s_test = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Global/test_global_recording_chunks.csv"
path_global_5s_val = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Global/val_global_recording_chunks.csv"

# Using pandas to read the CSV file into a DataFrame
df_train = pd.read_csv(path_global_5s_train)
df_test = pd.read_csv(path_global_5s_test)
df_val = pd.read_csv(path_global_5s_val)